## 1. Subscribe to the model package

To subscribe to the model package:
1. Open the model package listing page <font color='red'> For Seller to update:[Title_of_your_product](Provide link to your marketplace listing of your product).</font>
1. On the AWS Marketplace listing, click on the **Continue to subscribe** button.
1. On the **Subscribe to this software** page, review and click on **"Accept Offer"** if you and your organization agrees with EULA, pricing, and support terms. 
1. Once you click on **Continue to configuration button** and then choose a **region**, you will see a **Product Arn** displayed. This is the model package ARN that you need to specify while creating a deployable model using Boto3. Copy the ARN corresponding to your region and specify the same in the following cell.

## Resolve Oncology terminology with ICD-O Taxonomy

The International Classification of Diseases for Oncology (ICD-O) is a specialized system meticulously crafted for the classification and coding of neoplasms—those intricate entities we commonly refer to as tumors. ICD-O ensures consistent and precise reporting of cancer cases across various healthcare settings and research studies. By using standardized codes for tumor site, morphology, and behavior, it ensures accurate reporting and enables informed clinical management decisions.

- **Model**: `en.resolve.icdo_augmented`
- **Model Description**: This model is designed to map extracted clinical entities to specific ICD-O codes. It provides detailed code resolutions, top terms, and granular insights into body part references, enhancing precision in cancer-related documentation.


In [2]:
model_package_arn = "<Customer to specify Model package ARN corresponding to their AWS region>"

In [3]:
import base64
import json
import uuid
from sagemaker import ModelPackage
import sagemaker as sage
from sagemaker import get_execution_role
import boto3
from IPython.display import Image, display
from PIL import Image as ImageEdit
import numpy as np

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


In [4]:
sagemaker_session = sage.Session()
s3_bucket = sagemaker_session.default_bucket()
region = sagemaker_session.boto_region_name
account_id = boto3.client("sts").get_caller_identity().get("Account")
role = get_execution_role()

sagemaker = boto3.client("sagemaker")
s3_client = sagemaker_session.boto_session.client("s3")
ecr = boto3.client("ecr")
sm_runtime = boto3.client("sagemaker-runtime")

## 2. Create an endpoint and perform real-time inference

If you want to understand how real-time inference with Amazon SageMaker works, see [Documentation](https://docs.aws.amazon.com/sagemaker/latest/dg/how-it-works-hosting.html).

In [5]:
model_name = "en-resolve-icdo-augmented"

real_time_inference_instance_type = "ml.m4.xlarge"
batch_transform_inference_instance_type = "ml.m4.xlarge"


### A. Create an endpoint

In [6]:
# create a deployable model from the model package.
model = ModelPackage(
    role=role, model_package_arn=model_package_arn, sagemaker_session=sagemaker_session
)

# Deploy the model
predictor = model.deploy(1, real_time_inference_instance_type, endpoint_name=model_name)

---------!

Once endpoint has been created, you would be able to perform real-time inference.

In [7]:
import json
import pandas as pd
import os
import boto3

# Set display options
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

def process_data_and_invoke_realtime_endpoint(data, content_type, accept):

    content_type_to_format = {'application/json': 'json', 'application/jsonlines': 'jsonl'}
    input_format = content_type_to_format.get(content_type)
    if content_type not in content_type_to_format.keys() or accept not in content_type_to_format.keys():
        raise ValueError("Invalid content_type or accept. It should be either 'application/json' or 'application/jsonlines'.")

    i = 1
    input_dir = f'inputs/real-time/{input_format}'
    output_dir = f'outputs/real-time/{input_format}'
    s3_input_dir = f"{model_name}/validation-input/real-time/{input_format}"
    s3_output_dir = f"{model_name}/validation-output/real-time/{input_format}"

    input_file_name = f'{input_dir}/input{i}.{input_format}'
    output_file_name = f'{output_dir}/{os.path.basename(input_file_name)}.out'

    while os.path.exists(input_file_name) or os.path.exists(output_file_name):
        i += 1
        input_file_name = f'{input_dir}/input{i}.{input_format}'
        output_file_name = f'{output_dir}/{os.path.basename(input_file_name)}.out'

    os.makedirs(os.path.dirname(input_file_name), exist_ok=True)
    os.makedirs(os.path.dirname(output_file_name), exist_ok=True)

    input_data = json.dumps(data) if content_type == 'application/json' else data

    # Write input data to file
    with open(input_file_name, 'w') as f:
        f.write(input_data)

    # Upload input data to S3
    s3_client.put_object(Bucket=s3_bucket, Key=f"{s3_input_dir}/{os.path.basename(input_file_name)}", Body=bytes(input_data.encode('UTF-8')))

    # Invoke the SageMaker endpoint
    response = sm_runtime.invoke_endpoint(
        EndpointName=model_name,
        ContentType=content_type,
        Accept=accept,
        Body=input_data,
    )

    # Read response data
    response_data = json.loads(response["Body"].read().decode("utf-8")) if accept == 'application/json' else response['Body'].read().decode('utf-8')

    # Save response data to file
    with open(output_file_name, 'w') as f_out:
        if accept == 'application/json':
            json.dump(response_data, f_out, indent=4)
        else:
            for item in response_data.split('\n'):
                f_out.write(item + '\n')

    # Upload response data to S3
    output_s3_key = f"{s3_output_dir}/{os.path.basename(output_file_name)}"
    if accept == 'application/json':
        s3_client.put_object(Bucket=s3_bucket, Key=output_s3_key, Body=json.dumps(response_data).encode('UTF-8'))
    else:
        s3_client.put_object(Bucket=s3_bucket, Key=output_s3_key, Body=response_data)

    return response_data

### Initial Setup

In [8]:
docs = [
    "A few studies have demonstrated that ANGPTL1 functions as a tumor suppressor gene in breast cancer  , hepatocellular carcinoma  , colorectal cancer and parathyroid carcinoma.", 
    "TRAF6 is a putative oncogene in a variety of cancers including  bladder cancer , and skin cancer. WWP2 appears to regulate the expression of the well characterized tumor suppressor phosphatase and tensin homolog (PTEN)   in endometrial cancer   and squamous cell carcinoma.",
]


sample_text = "TRIM50 has only been shown to act as a tumor suppressor in hepatocellular carcinoma and ovary cancer. HOXA10 exerts an oncogenic role in several tumors endometrial adenocarcinoma ."

### JSON

#### Example 1

  **Input format**:
  
  
```json
{
    "text": "Single text document"
}
```

In [9]:
input_json_data = {"text": sample_text}

data =  process_data_and_invoke_realtime_endpoint(input_json_data, content_type="application/json" , accept="application/json" )
pd.DataFrame(data["predictions"])

,document,ner_chunk,begin,end,ner_label,ner_confidence,icdo_code,icdo_resolution,icdo_confidence
0,TRIM50 has only been shown to act as a tumor suppressor in hepatocellular carcinoma and ovary cancer. HOXA10 exerts an oncogenic role in several tumors endometrial adenocarcinoma .,tumor suppressor,39,54,Oncological,0.75685,8001/1,tumor cells,0.3394
1,TRIM50 has only been shown to act as a tumor suppressor in hepatocellular carcinoma and ovary cancer. HOXA10 exerts an oncogenic role in several tumors endometrial adenocarcinoma .,hepatocellular carcinoma,59,82,Oncological,0.8806,8576/3,hepatoid carcinoma,1.0000
2,TRIM50 has only been shown to act as a tumor suppressor in hepatocellular carcinoma and ovary cancer. HOXA10 exerts an oncogenic role in several tumors endometrial adenocarcinoma .,ovary cancer,88,99,Oncological,0.8954,8010/3-C56.9,"carcinoma, of ovary",0.3385
3,TRIM50 has only been shown to act as a tumor suppressor in hepatocellular carcinoma and ovary cancer. HOXA10 exerts an oncogenic role in several tumors endometrial adenocarcinoma .,HOXA10,102,107,Oncological,0.4475,C72.4,acoustic nerve,0.3350
4,TRIM50 has only been shown to act as a tumor suppressor in hepatocellular carcinoma and ovary cancer. HOXA10 exerts an oncogenic role in several tumors endometrial adenocarcinoma .,several,137,143,Modifier,0.4946,9982/3,rars,0.3400
5,TRIM50 has only been shown to act as a tumor suppressor in hepatocellular carcinoma and ovary cancer. HOXA10 exerts an oncogenic role in several tumors endometrial adenocarcinoma .,tumors endometrial adenocarcinoma,145,177,Oncological,0.8318,8384/3-C53.1,"adenocarcinoma, endocervical type, of exocervix",0.3335


#### Example 2

  **Input format**:
  
  
```json
{
    "text": [
        "Text document 1",
        "Text document 2",
        ...
    ]
}
```

In [10]:
input_json_data = {"text": docs}

data =  process_data_and_invoke_realtime_endpoint(input_json_data, content_type="application/json" , accept="application/json" )
pd.DataFrame(data["predictions"])

,document,ner_chunk,begin,end,ner_label,ner_confidence,icdo_code,icdo_resolution,icdo_confidence
0,"A few studies have demonstrated that ANGPTL1 functions as a tumor suppressor gene in breast cancer , hepatocellular carcinoma , colorectal cancer and parathyroid carcinoma.",tumor suppressor,60,75,Oncological,0.6416,8001/1,tumor cells,0.3394
1,"A few studies have demonstrated that ANGPTL1 functions as a tumor suppressor gene in breast cancer , hepatocellular carcinoma , colorectal cancer and parathyroid carcinoma.",breast cancer,85,97,Oncological,0.939,8140/3-C50.9,"adenocarcinoma, of breast",0.3384
2,"A few studies have demonstrated that ANGPTL1 functions as a tumor suppressor gene in breast cancer , hepatocellular carcinoma , colorectal cancer and parathyroid carcinoma.",hepatocellular carcinoma,101,124,Oncological,0.94219995,8576/3,hepatoid carcinoma,1.0000
3,"A few studies have demonstrated that ANGPTL1 functions as a tumor suppressor gene in breast cancer , hepatocellular carcinoma , colorectal cancer and parathyroid carcinoma.",colorectal cancer,128,144,Oncological,0.9151,8144/3-C18.9,"adenocarcinoma, intestinal type of colon",0.3367
4,"A few studies have demonstrated that ANGPTL1 functions as a tumor suppressor gene in breast cancer , hepatocellular carcinoma , colorectal cancer and parathyroid carcinoma.",parathyroid carcinoma,150,170,Oncological,0.8334,8010/3-C75.0,"carcinoma, of parathyroid gland",0.3379
5,"TRAF6 is a putative oncogene in a variety of cancers including bladder cancer , and skin cancer. WWP2 appears to regulate the expression of the well characterized tumor suppressor phosphatase and tensin homolog (PTEN) in endometrial cancer and squamous cell carcinoma.",cancers,45,51,Oncological,0.9532,8000/3,cancer,0.3755
6,"TRAF6 is a putative oncogene in a variety of cancers including bladder cancer , and skin cancer. WWP2 appears to regulate the expression of the well characterized tumor suppressor phosphatase and tensin homolog (PTEN) in endometrial cancer and squamous cell carcinoma.",bladder cancer,63,76,Oncological,0.90855,8010/3-C67.9,"carcinoma, of bladder",0.3374
7,"TRAF6 is a putative oncogene in a variety of cancers including bladder cancer , and skin cancer. WWP2 appears to regulate the expression of the well characterized tumor suppressor phosphatase and tensin homolog (PTEN) in endometrial cancer and squamous cell carcinoma.",skin cancer,84,94,Oncological,0.86455,8010/9-C44.9,carcinomatosis of skin,0.3417
8,"TRAF6 is a putative oncogene in a variety of cancers including bladder cancer , and skin cancer. WWP2 appears to regulate the expression of the well characterized tumor suppressor phosphatase and tensin homolog (PTEN) in endometrial cancer and squamous cell carcinoma.",tumor,163,167,Oncological,0.9767,8000/1,tumor,0.4155
9,"TRAF6 is a putative oncogene in a variety of cancers including bladder cancer , and skin cancer. WWP2 appears to regulate the expression of the well characterized tumor suppressor phosphatase and tensin homolog (PTEN) in endometrial cancer and squamous cell carcinoma.",endometrial cancer,221,238,Oncological,0.96150005,8380/3,endometrioid carcinoma,0.3409


### JSON Lines

  **Input format**:
  
```json
{"text": "Text document 1"}
{"text": "Text document 2"}
```

In [11]:
import json

def create_jsonl(records):
    json_records = []

    for text in records:
        record = {
            "text": text
        }
        json_records.append(record)

    json_lines = '\n'.join(json.dumps(record) for record in json_records)

    return json_lines

input_jsonl_data = create_jsonl(docs)

#### Example 1

In [12]:
data = process_data_and_invoke_realtime_endpoint(input_jsonl_data, content_type="application/jsonlines" , accept="application/jsonlines" )
print(data)

{"predictions": [{"ner_chunk": "tumor suppressor", "begin": 60, "end": 75, "ner_label": "Oncological", "ner_confidence": "0.6416", "icdo_code": "8001/1", "icdo_resolution": "tumor cells", "icdo_confidence": "0.3394"}, {"ner_chunk": "breast cancer", "begin": 85, "end": 97, "ner_label": "Oncological", "ner_confidence": "0.939", "icdo_code": "8140/3-C50.9", "icdo_resolution": "adenocarcinoma, of breast", "icdo_confidence": "0.3384"}, {"ner_chunk": "hepatocellular carcinoma", "begin": 101, "end": 124, "ner_label": "Oncological", "ner_confidence": "0.94219995", "icdo_code": "8576/3", "icdo_resolution": "hepatoid carcinoma", "icdo_confidence": "1.0000"}, {"ner_chunk": "colorectal cancer", "begin": 128, "end": 144, "ner_label": "Oncological", "ner_confidence": "0.9151", "icdo_code": "8144/3-C18.9", "icdo_resolution": "adenocarcinoma, intestinal type of colon", "icdo_confidence": "0.3367"}, {"ner_chunk": "parathyroid carcinoma", "begin": 150, "end": 170, "ner_label": "Oncological", "ner_confid

### C. Delete the endpoint

Now that you have successfully performed a real-time inference, you do not need the endpoint any more. You can terminate the endpoint to avoid being charged.

In [13]:
model.sagemaker_session.delete_endpoint(model_name)
model.sagemaker_session.delete_endpoint_config(model_name)

## 3. Batch inference

In [14]:
import json
import os

input_dir = 'inputs/batch'
json_input_dir = f"{input_dir}/json"
jsonl_input_dir = f"{input_dir}/jsonl"

output_dir = 'outputs/batch'
json_output_dir = f"{output_dir}/json"
jsonl_output_dir = f"{output_dir}/jsonl"

os.makedirs(json_input_dir, exist_ok=True)
os.makedirs(jsonl_input_dir, exist_ok=True)
os.makedirs(json_output_dir, exist_ok=True)
os.makedirs(jsonl_output_dir, exist_ok=True)

validation_json_file_name = "input.json"

validation_jsonl_file_name = "input.jsonl"

validation_input_json_path = f"s3://{s3_bucket}/{model_name}/validation-input/batch/json/"
validation_output_json_path = f"s3://{s3_bucket}/{model_name}/validation-output/batch/json/"

validation_input_jsonl_path = f"s3://{s3_bucket}/{model_name}/validation-input/batch/jsonl/"
validation_output_jsonl_path = f"s3://{s3_bucket}/{model_name}/validation-output/batch/jsonl/"

def write_and_upload_to_s3(input_data, file_name):
    file_format = os.path.splitext(file_name)[1].lower()
    if file_format == ".json":
        input_data = json.dumps(input_data)

    with open(file_name, "w") as f:
        f.write(input_data)

    s3_client.put_object(
        Bucket=s3_bucket,
        Key=f"{model_name}/validation-input/batch/{file_format[1:]}/{os.path.basename(file_name)}",
        Body=(bytes(input_data.encode("UTF-8"))),
    )

In [15]:
input_jsonl_data = create_jsonl(docs)
input_json_data = {"text": docs}

write_and_upload_to_s3(input_json_data, f"{json_input_dir}/{validation_json_file_name}")

write_and_upload_to_s3(input_jsonl_data, f"{jsonl_input_dir}/{validation_jsonl_file_name}")

### JSON

In [ ]:
# Initialize a SageMaker Transformer object for making predictions
transformer = model.transformer(
    instance_count=1,
    instance_type=batch_transform_inference_instance_type,
    accept="application/json",
    output_path=validation_output_json_path
)

transformer.transform(validation_input_json_path, content_type="application/json")
transformer.wait()

In [17]:
from urllib.parse import urlparse

def process_s3_json_output_and_save(validation_file_name):

    output_file_path = f"{json_output_dir}/{validation_file_name}.out"
    parsed_url = urlparse(transformer.output_path)
    file_key = f"{parsed_url.path[1:]}{validation_file_name}.out"
    response = s3_client.get_object(Bucket=s3_bucket, Key=file_key)

    data = json.loads(response["Body"].read().decode("utf-8"))
    df = pd.DataFrame(data["predictions"])
    display(df)

    # Save the data to the output file
    with open(output_file_path, 'w') as f_out:
        json.dump(data, f_out, indent=4)

In [18]:
process_s3_json_output_and_save(validation_json_file_name)

,document,ner_chunk,begin,end,ner_label,ner_confidence,icdo_code,icdo_resolution,icdo_confidence
0,"A few studies have demonstrated that ANGPTL1 functions as a tumor suppressor gene in breast cancer , hepatocellular carcinoma , colorectal cancer and parathyroid carcinoma.",tumor suppressor,60,75,Oncological,0.6416,8001/1,tumor cells,0.3394
1,"A few studies have demonstrated that ANGPTL1 functions as a tumor suppressor gene in breast cancer , hepatocellular carcinoma , colorectal cancer and parathyroid carcinoma.",breast cancer,85,97,Oncological,0.939,8140/3-C50.9,"adenocarcinoma, of breast",0.3384
2,"A few studies have demonstrated that ANGPTL1 functions as a tumor suppressor gene in breast cancer , hepatocellular carcinoma , colorectal cancer and parathyroid carcinoma.",hepatocellular carcinoma,101,124,Oncological,0.94219995,8576/3,hepatoid carcinoma,1.0000
3,"A few studies have demonstrated that ANGPTL1 functions as a tumor suppressor gene in breast cancer , hepatocellular carcinoma , colorectal cancer and parathyroid carcinoma.",colorectal cancer,128,144,Oncological,0.9151,8144/3-C18.9,"adenocarcinoma, intestinal type of colon",0.3367
4,"A few studies have demonstrated that ANGPTL1 functions as a tumor suppressor gene in breast cancer , hepatocellular carcinoma , colorectal cancer and parathyroid carcinoma.",parathyroid carcinoma,150,170,Oncological,0.8334,8010/3-C75.0,"carcinoma, of parathyroid gland",0.3379
5,"TRAF6 is a putative oncogene in a variety of cancers including bladder cancer , and skin cancer. WWP2 appears to regulate the expression of the well characterized tumor suppressor phosphatase and tensin homolog (PTEN) in endometrial cancer and squamous cell carcinoma.",cancers,45,51,Oncological,0.9532,8000/3,cancer,0.3755
6,"TRAF6 is a putative oncogene in a variety of cancers including bladder cancer , and skin cancer. WWP2 appears to regulate the expression of the well characterized tumor suppressor phosphatase and tensin homolog (PTEN) in endometrial cancer and squamous cell carcinoma.",bladder cancer,63,76,Oncological,0.90855,8010/3-C67.9,"carcinoma, of bladder",0.3374
7,"TRAF6 is a putative oncogene in a variety of cancers including bladder cancer , and skin cancer. WWP2 appears to regulate the expression of the well characterized tumor suppressor phosphatase and tensin homolog (PTEN) in endometrial cancer and squamous cell carcinoma.",skin cancer,84,94,Oncological,0.86455,8010/9-C44.9,carcinomatosis of skin,0.3417
8,"TRAF6 is a putative oncogene in a variety of cancers including bladder cancer , and skin cancer. WWP2 appears to regulate the expression of the well characterized tumor suppressor phosphatase and tensin homolog (PTEN) in endometrial cancer and squamous cell carcinoma.",tumor,163,167,Oncological,0.9767,8000/1,tumor,0.4155
9,"TRAF6 is a putative oncogene in a variety of cancers including bladder cancer , and skin cancer. WWP2 appears to regulate the expression of the well characterized tumor suppressor phosphatase and tensin homolog (PTEN) in endometrial cancer and squamous cell carcinoma.",endometrial cancer,221,238,Oncological,0.96150005,8380/3,endometrioid carcinoma,0.3409


### JSON Lines

In [ ]:
transformer = model.transformer(
    instance_count=1,
    instance_type=batch_transform_inference_instance_type,
    accept="application/jsonlines",
    output_path=validation_output_jsonl_path
)
transformer.transform(validation_input_jsonl_path, content_type="application/jsonlines")
transformer.wait()

In [20]:
from urllib.parse import urlparse

def process_s3_jsonlines_output_and_save(validation_file_name):

    output_file_path = f"{jsonl_output_dir}/{validation_file_name}.out"
    parsed_url = urlparse(transformer.output_path)
    file_key = f"{parsed_url.path[1:]}{validation_file_name}.out"
    response = s3_client.get_object(Bucket=s3_bucket, Key=file_key)

    data = response["Body"].read().decode("utf-8")
    print(data)

    # Save the data to the output file
    with open(output_file_path, 'w') as f_out:
        for item in data.split('\n'):
            f_out.write(item + '\n')

In [21]:
process_s3_jsonlines_output_and_save(validation_jsonl_file_name)

{"predictions": [{"ner_chunk": "tumor suppressor", "begin": 60, "end": 75, "ner_label": "Oncological", "ner_confidence": "0.6416", "icdo_code": "8001/1", "icdo_resolution": "tumor cells", "icdo_confidence": "0.3394"}, {"ner_chunk": "breast cancer", "begin": 85, "end": 97, "ner_label": "Oncological", "ner_confidence": "0.939", "icdo_code": "8140/3-C50.9", "icdo_resolution": "adenocarcinoma, of breast", "icdo_confidence": "0.3384"}, {"ner_chunk": "hepatocellular carcinoma", "begin": 101, "end": 124, "ner_label": "Oncological", "ner_confidence": "0.94219995", "icdo_code": "8576/3", "icdo_resolution": "hepatoid carcinoma", "icdo_confidence": "1.0000"}, {"ner_chunk": "colorectal cancer", "begin": 128, "end": 144, "ner_label": "Oncological", "ner_confidence": "0.9151", "icdo_code": "8144/3-C18.9", "icdo_resolution": "adenocarcinoma, intestinal type of colon", "icdo_confidence": "0.3367"}, {"ner_chunk": "parathyroid carcinoma", "begin": 150, "end": 170, "ner_label": "Oncological", "ner_confid

In [22]:
model.delete_model()

INFO:sagemaker:Deleting model with name: en-resolve-icdo-augmented-2024-05-02-08-25-11-377


### Unsubscribe to the listing (optional)

If you would like to unsubscribe to the model package, follow these steps. Before you cancel the subscription, ensure that you do not have any [deployable model](https://console.aws.amazon.com/sagemaker/home#/models) created from the model package or using the algorithm. Note - You can find this information by looking at the container name associated with the model. 

**Steps to unsubscribe to product from AWS Marketplace**:
1. Navigate to __Machine Learning__ tab on [__Your Software subscriptions page__](https://aws.amazon.com/marketplace/ai/library?productType=ml&ref_=mlmp_gitdemo_indust)
2. Locate the listing that you want to cancel the subscription for, and then choose __Cancel Subscription__  to cancel the subscription.

